<a href="https://colab.research.google.com/github/saptarshidatta96/MTech_Sem3/blob/main/NLU_ASSIGNMENT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import csv
import requests
import numpy as np
import xml.etree.ElementTree as ET
import random
import tensorflow as tf
from scipy.sparse import csr_matrix
import tensorflow as tf
from keras.preprocessing import sequence
from keras.preprocessing import text
from tensorflow import keras
from keras.models import Sequential
from keras import models
from keras.layers import *
from keras.models import *
from keras import backend as K
from keras import initializers
from keras import regularizers
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional, Attention, Flatten, Activation, RepeatVector, Permute, Multiply
from keras.layers import CuDNNLSTM
from keras.layers import Dropout, Multiply, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [4]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [5]:
import pandas as pd
import xml.etree.ElementTree as ET
import io
import xml.dom.minidom

def iter_docs(author):
  author_attr = author.attrib
  for doc in author.iter('sentence'):
    doc_dict = author_attr.copy()
    for text in doc.iter('text'):
      doc_dict['text'] = text.text
      for cat in doc.iter('aspectCategory'):
        doc_dict.update(cat.attrib)
      yield doc_dict


etree = ET.parse("/content/gdrive/MyDrive/train.xml")
doc_df = pd.DataFrame(list(iter_docs(etree.getroot())))

In [6]:
doc_df.head()

,text,category,polarity
0,It might be the best sit down food I've had in...,place,neutral
1,Hostess was extremely accommodating when we ar...,miscellaneous,neutral
2,We were a couple of minutes late for our reser...,staff,negative
3,"Though the service might be a little slow, the...",staff,positive
4,Although we arrived at the restaurant 10 min l...,miscellaneous,neutral


In [7]:
doc_df.shape

(3134, 3)

In [8]:
doc_df['category'].unique()

array(['place', 'miscellaneous', 'staff', 'food', 'ambience', 'service',
       'menu', 'price'], dtype=object)

In [9]:
doc_df['polarity'].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [10]:
df_category = pd.get_dummies(doc_df['category'], columns = ['food', 'place', 'staff', 'miscellaneous', 'service', 'price', 'menu', 'ambience'])

In [11]:
df_polarity = pd.get_dummies(doc_df['polarity'], columns = ['neutral', 'negative', 'positive'])

In [12]:
df = pd.concat([df_category, df_polarity], axis=1)

In [13]:
df = pd.concat([doc_df['text'], df], axis=1)

In [14]:
df.head()

,text,ambience,food,menu,miscellaneous,place,price,service,staff,negative,neutral,positive
0,It might be the best sit down food I've had in...,0,0,0,0,1,0,0,0,0,1,0
1,Hostess was extremely accommodating when we ar...,0,0,0,1,0,0,0,0,0,1,0
2,We were a couple of minutes late for our reser...,0,0,0,0,0,0,0,1,1,0,0
3,"Though the service might be a little slow, the...",0,0,0,0,0,0,0,1,0,0,1
4,Although we arrived at the restaurant 10 min l...,0,0,0,1,0,0,0,0,0,1,0


In [15]:
data = df['text']
data = list(data)
labels = df.drop(['text'], axis = 1)

In [16]:
def split_training_and_validation_sets(data, labels, validation_split):

    num_training_samples = int((1 - validation_split) * len(data))
    return ((data[:num_training_samples], labels[:num_training_samples]),
            (data[num_training_samples:], labels[num_training_samples:]))
    
(train_data, train_label), (valid_data, valid_label) = split_training_and_validation_sets(data, labels, 0.1)


In [17]:
print(valid_label)

      ambience  food  menu  miscellaneous  ...  staff  negative  neutral  positive
2820         0     1     0              0  ...      0         0        0         1
2821         0     0     0              0  ...      0         1        0         0
2822         0     0     1              0  ...      0         0        1         0
2823         0     1     0              0  ...      0         1        0         0
2824         0     1     0              0  ...      0         0        1         0
...        ...   ...   ...            ...  ...    ...       ...      ...       ...
3129         0     0     0              0  ...      1         1        0         0
3130         0     0     1              0  ...      0         0        1         0
3131         0     0     0              0  ...      0         0        1         0
3132         0     0     0              0  ...      0         0        0         1
3133         0     0     0              0  ...      1         1        0         0

[31

In [18]:
train_label = tf.convert_to_tensor(train_label, dtype=tf.float32)
valid_label = tf.convert_to_tensor(valid_label, dtype=tf.float32)

In [19]:
def sequence_vectorizer(train_data, valid_data):

    # Create vocabulary with training texts.
    tokenizer = text.Tokenizer(num_words=20000)
    tokenizer.fit_on_texts(train_data)

    # Vectorize training and validation texts.
    x_train = tokenizer.texts_to_sequences(train_data)
    x_val = tokenizer.texts_to_sequences(valid_data)

    # Get max sequence length.
    max_length = len(max(x_train, key=len))
    if max_length > 50:
        max_length = 50

    # Fix sequence length to max value. Sequences shorter than the length are
    # padded in the beginning and sequences longer are truncated
    # at the beginning.
    x_train = sequence.pad_sequences(x_train, maxlen=max_length)
    x_val = sequence.pad_sequences(x_val, maxlen=max_length)

    x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
    x_val = tf.convert_to_tensor(x_val, dtype=tf.float32)

    return x_train, x_val, tokenizer.word_index

In [20]:
x_train, x_val, word_index = sequence_vectorizer(train_data, valid_data)

In [21]:
len(word_index)

6269

In [22]:
print(x_train.shape)

(2820, 50)


In [23]:
!unzip "/content/gdrive/MyDrive/glove.42B.300d.zip" -d "/content"

Archive:  /content/gdrive/MyDrive/glove.42B.300d.zip
  inflating: /content/glove.42B.300d.txt  


In [32]:
glove_file = "glove.42B.300d.txt"
import tqdm

EMBEDDING_VECTOR_LENGTH = 50 # <=200
def construct_embedding_matrix(glove_file, word_index):
    embedding_dict = {}
    with open(glove_file,'r') as f:
        for line in f:
            values=line.split()
            # get the word
            word=values[0]
            if word in word_index.keys():
                # get the vector
                vector = np.asarray(values[1:], 'float32')
                embedding_dict[word] = vector
    ###  oov words (out of vacabulary words) will be mapped to 0 vectors

    num_words=len(word_index)+1
    #initialize it to 0
    embedding_matrix=np.zeros((num_words, EMBEDDING_VECTOR_LENGTH))

    for word,i in tqdm.tqdm(word_index.items()):
        if i < num_words:
            vect=embedding_dict.get(word, [])
            if len(vect)>0:
                embedding_matrix[i] = vect[:EMBEDDING_VECTOR_LENGTH]
    return embedding_matrix
  
embedding_matrix =  construct_embedding_matrix(glove_file, word_index)

100%|██████████| 6269/6269 [00:00<00:00, 359032.33it/s]


In [25]:
embedding_matrix.shape

(6270, 50)

In [26]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [95]:
class attention(Layer):
    
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences
        super(attention,self).__init__()
        
    def build(self, input_shape):
        
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")
        
        super(attention,self).build(input_shape)
        
    def call(self, x):
        
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        
        if self.return_sequences:
            return output
        
        return K.sum(output, axis=1)

In [104]:
lstm_out = 200
model = Sequential()
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_VECTOR_LENGTH,
                            weights=[embedding_matrix],
                            input_length=50,
                            trainable=False)


model.add(embedding_layer)
#model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Bidirectional(LSTM(lstm_out,return_sequences=True)))
model.add(Bidirectional(LSTM(lstm_out,return_sequences=True)))
model.add(Bidirectional(LSTM(lstm_out,return_sequences=True)))
#model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(attention(return_sequences=True))


model.add(Flatten())
model.add(Dropout(0.3))

model.add(Dense(11, activation='sigmoid'))


opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_34 (Embedding)    (None, 50, 50)            313500    
                                                                 
 bidirectional_39 (Bidirecti  (None, 50, 400)          401600    
 onal)                                                           
                                                                 
 bidirectional_40 (Bidirecti  (None, 50, 400)          961600    
 onal)                                                           
                                                                 
 bidirectional_41 (Bidirecti  (None, 50, 400)          961600    
 onal)                                                           
                                                                 
 attention_11 (attention)    (None, 50, 400)           450       
                                                     

In [101]:
callbacks = [EarlyStopping(monitor='val_accuracy', patience=5)]

In [105]:
model.fit(x_train, train_label,
            epochs=50,
            #callbacks=callbacks,
            validation_data=(x_val, valid_label),
            verbose=2,
            batch_size=128)

Epoch 1/50
23/23 - 16s - loss: 0.4447 - accuracy: 0.0691 - val_loss: 0.4121 - val_accuracy: 0.0000e+00 - 16s/epoch - 685ms/step
Epoch 2/50
23/23 - 3s - loss: 0.4165 - accuracy: 0.0000e+00 - val_loss: 0.4106 - val_accuracy: 0.0000e+00 - 3s/epoch - 128ms/step
Epoch 3/50
23/23 - 3s - loss: 0.4172 - accuracy: 0.0000e+00 - val_loss: 0.4105 - val_accuracy: 0.0000e+00 - 3s/epoch - 127ms/step
Epoch 4/50
23/23 - 3s - loss: 0.4153 - accuracy: 0.0000e+00 - val_loss: 0.4086 - val_accuracy: 0.0000e+00 - 3s/epoch - 127ms/step
Epoch 5/50
23/23 - 3s - loss: 0.4163 - accuracy: 0.0000e+00 - val_loss: 0.4074 - val_accuracy: 0.0000e+00 - 3s/epoch - 128ms/step
Epoch 6/50
23/23 - 3s - loss: 0.4135 - accuracy: 0.0000e+00 - val_loss: 0.4229 - val_accuracy: 0.0000e+00 - 3s/epoch - 127ms/step
Epoch 7/50
23/23 - 3s - loss: 0.4172 - accuracy: 0.0000e+00 - val_loss: 0.4057 - val_accuracy: 0.0000e+00 - 3s/epoch - 128ms/step
Epoch 8/50
23/23 - 3s - loss: 0.4105 - accuracy: 0.0000e+00 - val_loss: 0.4132 - val_accurac